## Extracting information from a text using an LLM

As an LLM "understands" a language, it can be suited for tasks like sentiment analysis or information extraction.

In this Notebook, we are going to use our LLM to analyze the claims to find the state of mind of the person writing, and the location and time of the accident.

### Requirements and Imports

If you have selected the right workbench image to launch as per the Lab's instructions, you should already have all the needed libraries. If not uncomment the first line in the next cell to install all the right packages.

In [ ]:
# !pip install --no-cache-dir --no-dependencies --disable-pip-version-check -r requirements.txt # Uncomment only if you have not selected the right workbench image

import json
import os
from os import listdir
from os.path import isfile, join
from langchain.chains import LLMChain
from langchain.llms import HuggingFaceTextGenInference
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

### Langchain pipeline

Again, we are going to use Langchain to define our task pipeline.

In [ ]:
# LLM Inference Server URL
inference_server_url = "http://llm.ic-shared-llm.svc.cluster.local:3000/"

# LLM definition
llm = HuggingFaceTextGenInference(
    inference_server_url=inference_server_url,
    max_new_tokens=512,
    top_k=10,
    top_p=0.95,
    typical_p=0.95,
    temperature=0.01,
    repetition_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

We will now define a **template** to use specifically with the different tasks.

In [ ]:
template="""<s>[INST]
You are a helpful, respectful and honest assistant.
Always assist with care, respect, and truth. Respond with utmost utility yet securely.
Avoid harmful, unethical, prejudiced, or negative content. Ensure replies promote fairness and positivity.
I will give you a text, then ask a question about it. Give a precise and as concise as possible answer to this question.

### TEXT:
{text}

### QUESTION:
{query}

### ANSWER:
[/INST]
"""
PROMPT = PromptTemplate(input_variables=["input"], template=template)

And we can now create the **conversation** object that we will use to query the model.

In [ ]:
conversation = LLMChain(llm=llm,
                        prompt=PROMPT,
                        verbose=False
                        )

We are now ready to query the model!

In the `claims` folder we have JSON files with examples of claims that could be received. We are going to read those files, display them, then the analysis that the LLM made.

In [ ]:
# Read the claims and populate a dictionary

claims_path = 'claims'
onlyfiles = [f for f in listdir(claims_path) if isfile(join(claims_path, f))]

claims = {}

for filename in onlyfiles:
    # Opening JSON file
    with open(os.path.join(claims_path, filename), 'r') as file:
        data = json.load(file)
    claims[filename] = data

In [ ]:
# Analyze the claims

for filename in onlyfiles:
    print(f"***************************")
    print(f"* Claim: {filename}")
    print(f"***************************")
    print("Original content:")
    print("-----------------")
    print(f"Subject: {claims[filename]['subject']}\nContent:\n{claims[filename]['content']}\n\n")
    print('Analysis:')
    print("--------")
    text_input = f"Subject: {claims[filename]['subject']}\nContent:\n{claims[filename]['content']}"
    sentiment_query = "What is the sentiment of the person sending this claim?"
    location_query = "Where does the event the claim is related to happen?"
    time_query = "When does the event the claim is related to happen? If possible, specify the date and the time."
    print(f"- Sentiment: ")
    conversation.predict(text=text_input, query=sentiment_query);
    print("\n- Location: ")
    conversation.predict(text=text_input, query=location_query);
    print("\n- Time: ")
    conversation.predict(text=text_input, query=time_query);
    print("\n\n                          ----====----\n")

We will come back to this notebook at section 3.5 for some exercises, so you can leave it open for the moment.